# Mod 5 Project - Using Machine Learning to Predict Coronavirus

This project uses daily updated coronavirus data from John Hopkins Hospital

## COVID-19 MAP

### HOW MANY COUNTRIES, HOW MANY PEOPLE CONTAGIOUS, HOW MANY DEATH CASES???

## PERCENTAGE OF CONFIRMED CASES/POPULATION, DEATH CASES/CONFIRMED CASES, RECOVERED/CONFIRMED CASES.

### STEP 1: IMPORT ALL NECCESSARY LIBRARIES

In [1]:
# import all neccessary libraries
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta

import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf # for determining (p,q) orders
from statsmodels.tsa.seasonal import seasonal_decompose      # for ETS Plots
from pmdarima import auto_arima                              # for determining ARIMA orders

from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import itertools

sns.set_style('whitegrid')
plt.style.use('ggplot')

# ignore all harmless waring to keep the notebook clean
warnings.filterwarnings('ignore')

# keep the plot inline in notebookb
%matplotlib inline

In [4]:
### from __future__ import print_function
import pandas as pd
import numpy as np
import os
import pickle
import os.path
from datetime import datetime
import pyarrow
import matplotlib.pyplot as plt
%matplotlib inline
font = {'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)

#set ggplot style
plt.style.use('ggplot')


In [5]:
# Dynamic parameters
data_dir  = './data/' + str(datetime.date(datetime.now()))
agg_file  = 'agg_data_{}.parquet.gzip'.format(datetime.date(datetime.now()))
trend_file  = 'trend_{}.csv'.format(datetime.date(datetime.now()))
report  = 'report_{}.xlsx'.format(datetime.date(datetime.now()))

COUNTRY = 'Uk'


print(trend_file)

# import data
agg_df = pd.read_parquet(os.path.join(data_dir, agg_file))
daily_df = pd.read_csv(os.path.join(data_dir, trend_file))

# daily_df.new_confirmed_cases = daily_df.new_confirmed_cases.abs()

#Create place to save diagrams
image_dir = './images/'
if not os.path.exists(image_dir):
    os.mkdir(image_dir)


trend_2020-03-26.csv


OSError: Passed non-file path: ./data/2020-03-26/agg_data_2020-03-26.parquet.gzip

### STEP 2: LOAD DATA
#### We have 3 data sets: Confirmed cases, Death cases & Recovered cases

In [ ]:
# load data
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'
df_confirmed = pd.read_csv(url + 'time_series_19-covid-Confirmed.csv')
df_deaths = pd.read_csv(url + 'time_series_19-covid-Deaths.csv')
df_recovered = pd.read_csv(url + 'time_series_19-covid-Recovered.csv')

In [ ]:
df_population_K = pd.read_csv('population_by_country_2020.csv')
df_population_K.head()

In [ ]:
df_population_K.info()

In [ ]:
df_2020_K = df_population_K.filter(['Country (or dependency)', 'Population (2020)'])
df_2020_K.reset_index(inplace=True)
df_2020_K.drop(['index'], axis=1, inplace=True)
df_2020_K.head()


### STEP 3: INITIAL EXPLORATORY DATA ANALYSIS - EDA:

### A/ INITIAL ANALYSIS - df_confirmed

In [ ]:
df_confirmed.head()

In [ ]:
df_confirmed.tail()

In [ ]:
df_confirmed.describe(include=['object', 'bool'])

In [ ]:
df_confirmed.describe()

In [ ]:
# count the null columns
null_columns = df_confirmed.columns[df_confirmed.isnull().any()]
df_confirmed[null_columns].isnull().sum().head()

In [ ]:
# glance look at the null data
pd.set_option('display.max_columns', 999)
print(df_confirmed[df_confirmed.isnull().any(axis=1)][null_columns].head())

In [ ]:
# glance look at the null data
pd.set_option('display.max_rows', 999)
print(df_confirmed[df_confirmed.isnull().any(axis=1)][null_columns])

In [ ]:
len(df_confirmed['Province/State'].unique())

In [ ]:
print(df_confirmed['Province/State'].unique())

In [ ]:
len(df_confirmed['Country/Region'].unique())

In [ ]:
print(df_confirmed['Country/Region'].unique())

### B/ INITIAL ANALYSIS - df_deaths

In [ ]:
df_deaths.head()

In [ ]:
df_deaths.tail()

In [ ]:
df_deaths.describe(include=['object', 'bool'])

In [ ]:
df_deaths.describe()

In [ ]:
# count the null columns
null_columns = df_deaths.columns[df_deaths.isnull().any()]
df_deaths[null_columns].isnull().sum().head()

In [ ]:
# glance look at the null data
pd.set_option('display.max_columns', 999)
print(df_deaths[df_deaths.isnull().any(axis=1)][null_columns].head())

In [ ]:
# glance look at the null data
pd.set_option('display.max_rows', 999)
print(df_deaths[df_deaths.isnull().any(axis=1)][null_columns])

In [ ]:
len(df_deaths['Province/State'].unique())

In [ ]:
print(df_deaths['Province/State'].unique())

In [ ]:
len(df_deaths['Country/Region'].unique())

In [ ]:
print(df_deaths['Country/Region'].unique())

### C/ INITIAL ANALYSIS - df_recovered

In [ ]:
df_recovered.head()

In [ ]:
df_recovered.head()

In [ ]:
df_recovered.tail()

In [ ]:
df_recovered.describe(include=['object', 'bool'])

In [ ]:
df_recovered.describe()

In [ ]:
# count the null columns
null_columns = df_recovered.columns[df_recovered.isnull().any()]
df_recovered[null_columns].isnull().sum().head()

In [ ]:
# glance look at the null data
pd.set_option('display.max_columns', 999)
print(df_recovered[df_recovered.isnull().any(axis=1)][null_columns].head())

In [ ]:
# glance look at the null data
pd.set_option('display.max_rows', 999)
print(df_recovered[df_recovered.isnull().any(axis=1)][null_columns])

In [ ]:
len(df_recovered['Province/State'].unique())

In [ ]:
print(df_recovered['Province/State'].unique())

In [ ]:
len(df_recovered['Country/Region'].unique())

In [ ]:
print(df_recovered['Country/Region'].unique())

In [ ]:
covid19_list = list(df_recovered['Country/Region'].unique())
covid19_list

In [ ]:
covid19_population_K = df_2020_K[df_2020_K['Country (or dependency)'].isin(covid19_list)]
covid19_population_K.head()

In [ ]:
covid19_population_K.info()

### STEP 4: CLEANING DATA, REDUCE UNNECCESSARY & MISSING DATA FEATURES

#### The column 'Province/State' has 143 missing data, and we actually focus on country information not detail to Province/State. We are going reduce the Province/State column and using total cases by country.

In [ ]:
df_confirmed.head()

In [ ]:
# drop columns that we don't need them
# we focus on data by Country/Region only
dropped_df_confirmed = df_confirmed.drop(['Province/State','Lat', 'Long'], axis=1)
dropped_df_confirmed.head()

In [ ]:
dropped_df_confirmed.describe(include=['object', 'bool'])

In [ ]:
dropped_df_confirmed_total = dropped_df_confirmed.groupby(['Country/Region']).sum()
dropped_df_confirmed_total.head()

In [ ]:
dropped_df_confirmed_total.tail()

In [ ]:
dropped_df_confirmed_total.describe()


In [ ]:
dropped_df_confirmed.describe()

In [ ]:
#dropped_df_confirmed_total.describe(include=['object', 'bool'])

In [ ]:
dropped_df_confirmed_total["current_cases"] = dropped_df_confirmed_total.iloc[:, -1]
dropped_df_confirmed_total = dropped_df_confirmed_total[ ['current_cases'] + [ col for col in dropped_df_confirmed_total.columns if col != 'current_cases' ] ]

In [ ]:
sorted_df_confirmed=dropped_df_confirmed_total.drop(['current_cases'], axis=1)
sorted_df_confirmed

In [ ]:
dropped_df_confirmed_total.sort_values(by='current_cases', ascending=False, inplace = True)

In [ ]:
top30_confirmed = dropped_df_confirmed_total.iloc[0:30,:]
top30_confirmed

In [ ]:
top30_confirmed.reset_index(inplace = True)

In [ ]:
top_confirmed_list = top30_confirmed['Country/Region']
top_confirmed_list

In [ ]:
df_deaths.head()

In [ ]:
# drop columns that we don't need them
# we focus on data by Country/Region only
dropped_df_deaths = df_deaths.drop(['Province/State','Lat', 'Long'], axis=1)
dropped_df_deaths.head()

In [ ]:
dropped_df_deaths.describe(include=['object', 'bool'])

In [ ]:
dropped_df_deaths_total = dropped_df_deaths.groupby(['Country/Region']).sum()
dropped_df_deaths_total.head()

In [ ]:
dropped_df_deaths_total.tail()

In [ ]:
dropped_df_deaths_total.describe()


In [ ]:
dropped_df_deaths.describe()

In [ ]:
dropped_df_deaths_total["current_deaths"] = dropped_df_deaths_total.iloc[:, -1]
dropped_df_deaths_total = dropped_df_deaths_total[ ['current_deaths'] + [ col for col in dropped_df_deaths_total.columns if col != 'current_deaths' ] ]

In [ ]:
sorted_df_deaths=dropped_df_deaths_total.drop(['current_deaths'], axis=1)
sorted_df_deaths

In [ ]:
dropped_df_deaths_total.sort_values(by='current_deaths', ascending=False, inplace = True)

In [ ]:
top30_deaths = dropped_df_deaths_total.iloc[0:30,:]
top30_deaths

In [ ]:
top30_deaths.reset_index(inplace = True)

In [ ]:
top_deaths_list = top30_deaths['Country/Region']
top_deaths_list

In [ ]:
# df9_population = df_2020_K[df_2020_K['Country (or dependency)'].isin(top_deaths_list)]
# df9_population.head()

In [ ]:
# dropped_df_deaths_total.sort_values(by='current_deaths', ascending=False, inplace = True)

# top30_deaths = dropped_df_deaths_total.iloc[0:30,:]
# top30_deaths

# top30_deaths.reset_index(inplace = True)

# sns.set_style('whitegrid')
# g = sns.catplot(x='Country/Region', y='current_deaths', data=top30_deaths,
#                 kind='bar', palette='cool', height=6, aspect=2.5, legend = False)
# plt.legend(loc='upper right');
# g.set_xticklabels(rotation=45);

In [ ]:
dropped_df_deaths_total.head()

In [ ]:
df_recovered.head()

In [ ]:
# drop columns that we don't need them
# we focus on data by Country/Region only
dropped_df_recovered = df_recovered.drop(['Province/State','Lat', 'Long'], axis=1)
dropped_df_recovered.head()

In [ ]:
dropped_df_recovered_total = dropped_df_recovered.groupby(['Country/Region']).sum()
dropped_df_recovered_total.head()

In [ ]:
dropped_df_recovered_total.tail()

In [ ]:
dropped_df_recovered_total.describe()


In [ ]:
dropped_df_recovered.describe()

In [ ]:
dropped_df_recovered_total["current_recovered"] = dropped_df_recovered_total.iloc[:, -1]
dropped_df_recovered_total = dropped_df_recovered_total[ ['current_recovered'] + [ col for col in dropped_df_recovered_total.columns if col != 'current_recovered' ] ]

In [ ]:
sorted_df_recovered=dropped_df_recovered_total.drop(['current_recovered'], axis=1)
sorted_df_recovered

In [ ]:
dropped_df_recovered_total.sort_values(by='current_recovered', ascending=False, inplace = True)

In [ ]:
top30_recovered = dropped_df_recovered_total.iloc[0:30,:]
top30_recovered

In [ ]:
top30_recovered.reset_index(inplace = True)

In [ ]:
top_recovered_list = top30_recovered['Country/Region']
top_recovered_list

In [ ]:
df1 = dropped_df_confirmed_total.filter(['current_cases'])
df1.reset_index(inplace=True)
df1.head()

In [ ]:
df2 = dropped_df_deaths_total.filter(['current_deaths'])
df2.reset_index(inplace=True)
df2.head()

In [ ]:
df3 = dropped_df_recovered_total.filter(['current_recovered'])
df3.reset_index(inplace=True)
df3.head()

In [ ]:
covid19_population_K.head()

In [ ]:
covid19_population_K.info()

In [ ]:
covid19_population_K.rename(columns = {'Country (or dependency)':'Country/Region'}, inplace = True) 
covid19_population_K.head()

In [ ]:
df4 = pd.merge(df1, df2, on=['Country/Region'])
df5 = pd.merge(df3, df4, on=['Country/Region'])
df5_pop = pd.merge(df5, covid19_population_K, on=['Country/Region'])
df5_pop['Confirmed Percentage'] = (df5_pop['current_cases'] / df5_pop['Population (2020)']) *100
df5_pop['Recovered Percentage'] = (df5_pop['current_recovered'] / df5_pop['Population (2020)']) *100
df5_pop['Deaths Percentage'] = (df5_pop['current_deaths'] / df5_pop['Population (2020)']) *100

#df5.head()
df5_pop.head()

In [ ]:
df5_per = df5_pop.filter(['Country/Region','Confirmed Percentage', 'Recovered Percentage', 'Deaths Percentage'])
df5_per.head()

In [ ]:
df6 = df5.sort_values(['current_cases', 'current_deaths', 'current_recovered'], ascending=False)
df6.head()

# USING PERCENTAGE OF CASES PER POPULATION

In [ ]:
df7 = pd.melt(df5_per, id_vars=['Country/Region'], value_vars=['Confirmed Percentage', 'Deaths Percentage', 'Recovered Percentage'], 
              var_name='Types', value_name='Number Of Cases')
df7.info()
df7.head()

In [ ]:
df8 = df7[df7['Country/Region'].isin(top_confirmed_list) ]
df8.head()

In [ ]:
df9 = df7[df7['Country/Region'].isin(top_deaths_list) ]
df9.head()

sns.set_style('whitegrid')
g = sns.catplot(data=df9, x='Country/Region', y='Number Of Cases',
                           hue='Types', kind="bar", palette='cool', height=5, aspect = 2.5, legend = False)
plt.title('TOP 30 COUNTRIES OF DEATHS CASES ', size=16)
plt.legend(loc='upper right');
g.set_xticklabels(rotation=45);

In [ ]:
# df8_population = df_2020[df_2020['Location'].isin(top_confirmed_list)]
# df8_population.head()

In [ ]:
sns.set_style('whitegrid')
g = sns.catplot(data=df8, x='Country/Region', y='Number Of Cases',
                           hue='Types', kind="bar", palette='hot', height=5, aspect = 2.5, legend = False)
plt.title('TOP 30 COUNTRIES OF CONFIRMED CASES ', size=16)
plt.legend(loc='upper right');
g.set_xticklabels(rotation=45);

In [ ]:
# df10_population = df_2020[df_2020['Location'].isin(top_recovered_list)]
# df10_population.head()

In [ ]:
df10 = df7[df7['Country/Region'].isin(top_recovered_list) ]
df10.head()

sns.set_style('whitegrid')
g = sns.catplot(data=df10, x='Country/Region', y='Number Of Cases',
                           hue='Types', kind="bar", palette='autumn', height=5, aspect = 2.5, legend = False)
plt.title('TOP 30 COUNTRIES OF RECOVERED CASES ', size=16)
plt.legend(loc='upper right');
g.set_xticklabels(rotation=45);

In [ ]:
# top30_recovered.reset_index(inplace = True)
# sns.set_style('whitegrid')
# g = sns.catplot(x='Country/Region', y='current_recovered', data=top30_recovered,
#                 kind='bar', palette='autumn', height=6, aspect=2.5, legend = False)
# plt.legend(loc='upper right');
# g.set_xticklabels(rotation=45);

### STEP 5: RESHAPE FROM WIDE TO LONG FORMAT

In [ ]:
def melt_data(df):
    """
    melt data of one zip code from wide format to long format
    """
    
    melted = pd.melt(df, id_vars=['Country/Region'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    #melted = melted.dropna(subset=['value'])
    
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
def melt_df(df):
    """
    Loop through all zipcodes to melt data of each zipcode.
    Then, merge all melted data back together
    """
    
    merged = []
    for country in df['Country/Region']:
        melted = melt_data(df.loc[df['Country/Region'] == country])
        row = df.loc[df['Country/Region'] == country].iloc[:,:1]
        rows = pd.concat([row]*len(melted), ignore_index=True)
        merge = pd.concat([rows, melted.reset_index()], axis= 1)
        merged.append(merge)
    melted_df = pd.concat(merged)
    return melted_df

In [ ]:
sorted_df_confirmed.reset_index(inplace=True)

In [ ]:
sorted_df_confirmed.head()

In [ ]:
df = melt_df(sorted_df_confirmed)

In [ ]:
df.head()

In [ ]:
# check any columns has na/nan value, if there is missing data
#### forward fill missing value
#df['time'] = df['time'].ffill()
df.isnull().any()

#### Convert to Time Series Data by setting the time column as the index

In [ ]:
# make it as time series
df.set_index('time', inplace=True)

In [ ]:
# look at the head again
df.head()

In [ ]:
# check any columns has na/nan value, if there is missing data
df.isnull().any()

#### Now we see no more missing value in any column. Change the name of feature. Take a look at the final dataframe before performing EDA

In [ ]:
# rename columns
df.rename(columns={'Country/Region': 'Country'}, inplace=True)
df.head()

In [ ]:
df.tail()

In [ ]:
df

# Step 2: EDA and Visualization

#### Visualization of data at a glance, just to see the whole picture of corona virus spreading.

In [ ]:
# countries affected
countries = df['Country'].unique()
len(countries)

In [ ]:
plt.figure(figsize=(20,12))

for c in countries:
    df[df['Country']==c]['value'].plot(label=c)

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=10)
plt.show()

### From a quick plot above, we see that the corona virus had started and spreaded widely from China since Jan 22, 2020. There are some countries had started slightly from Feb 20 and the corona virus were actually growing much faster over time. Jan 26, 2020 the spreading speed has increased significantly in some countries such as Singapore, Spain, Iran, Italy . Mar 10, 2020 the corona virus has been spreading out crazily in other countries while there is no new confirmed case. 

#### Run an ETS Decomposition

In [ ]:
rcParams['figure.figsize'] = 12,5

for country in top_confirmed_list[0:10]:
    results = seasonal_decompose(df.loc[df['Country'] == country].value, model='add')
    fig = results.plot();
    fig.text(0.5, 1, f'COUNTRY: {country}')

#### Plot ACF and PACF of some zipcodes to check corelation

In [ ]:
fig = plt.figure(figsize=(20,40))
i = 0
lags=40
for country in top_confirmed_list[0:10]:
    i += 1
    ax = plt.subplot(10,2,i)
    title = f'Autocorrelation: Country: {country}'
    plot_acf(df.loc[df['Country'] == country].value, alpha=0.05, title=title, lags=lags, ax=ax);

    i += 1
    ax = plt.subplot(10,2,i)
    title=f'Partial Autocorrelation: Country: {country}'
    plot_pacf(df.loc[df['Country'] == country].value, alpha=0.05, title=title, lags=lags, ax=ax);

## Automate the Augmented Dickey-Fuller Test
##### Function that performs the augmented Dickey-Fuller Test to determine if an incoming time series is stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print("==============================================================")
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print("--------------------------------------------------------------")
    print(out.to_string())
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")
        
    print("--------------------------------------------------------------")

In [ ]:
# check ADF test for some zipcodes
for country in top_confirmed_list[0:10]:
    adf_test(df.loc[df['Country'] == country].value, title= f'COUNTRY: {country}')

### With the Augmented Dickey-Fuller Test results, all data for all zipcodes are non-staytionary. except Italy

# STEP 8: Explore ARIMA Modeling to Find Best Fit Model

### Use grid search to find best fit ARIMA model of one sample country

In [ ]:
# One sample test zipcode
country = top_confirmed_list[0]
country

In [ ]:
# test with one zipcode
result = auto_arima(df.loc[df['Country'] == country].value, 
                    start_p=1, start_q=1, max_p=3, max_q=3, m=12, start_P=0, seasonal=False, d=None, D=1, 
                    trace=False, error_action='ignore',suppress_warnings=True, stepwise=True)

result.summary()

### After having the best-fit model from auto_arima grid search, we double check with visualization of ETS decomposition and ACF and PACF

In [ ]:
rcParams['figure.figsize'] = 12,5
results = seasonal_decompose(df.loc[df['Country'] == country].value, model='add')
fig = results.plot();
fig.text(0.5,1,f'COUNTRY: {country}')

In [ ]:
plot_acf(df.loc[df['Country'] == country].value,alpha=0.05,title='ACF',lags=40);

In [ ]:
plot_pacf(df.loc[df['Country'] == country].value,alpha=0.05,title='PACF',lags=40);

#### Again, the ACF & PACF plots confirm the zipcode has none seasonality.

### Now, we wrote code to find best-fit models for a list of countries and store the fitted models in to a DataFrame for later dispaly and use
#### For quick testing purpose we only need to test with one country, but we can do for all countries

In [ ]:
# find the best fit model for the first test country
models = []
for country in top_confirmed_list[0:1]:
    result = auto_arima(df.loc[df['Country'] == country].value, 
                    start_p=1, start_q=1, max_p=3, max_q=3, m=12, start_P=0, seasonal=False, d=None, D=1, 
                    trace=False, error_action='ignore',suppress_warnings=True, stepwise=True)
    if result._is_seasonal():
        model = {'Country': country, 'model': 'SARIMAX', 'order': result.order,
                                     'seasonal_order': result.seasonal_order}
    else:
        model = {'Country': country, 'model': 'ARIMA', 'order': result.order, 
                                     'seasonal_order': None}
        
    models.append(model)
    
# convert models list into DataFrame for easy reading
model_df = pd.DataFrame(models, columns=['zipcode', 'model', 'order', 'seasonal_order'])
model_df

#### After having the model, we split data of this country into train/test dataset to evaluate the accurateness of the model

In [ ]:
first_country_data = df[df['Country']==country]
first_country_data.head()

In [ ]:
# get tail data of 3 year - test data set
first_country_data.tail(36)

In [ ]:
# Calculate the split time point for train & test data.
now = datetime.now()
start_date = datetime(2020, 1, 22)
end_date = now
delta = end_date - start_date
train_days = round(delta.days * 0.7)
split_date = start_date + timedelta(days=train_days)

start_date = start_date.date()
end_date = end_date.date()
split_date = split_date.date()

print(start_date)
print(split_date)
print(end_date)

In [ ]:
# split train/test data
#train_df = first_country_data.iloc[:train_days] #
train_df = first_country_data.loc[:split_date]
#test_df = first_country_data.iloc[split_date:]  #
test_start_date = split_date + timedelta(days=1) 
test_df = first_country_data.loc[test_start_date:]

In [ ]:
train_df.tail()

In [ ]:
test_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df

In [ ]:
test_df.info()

In [ ]:
# fit the best model with full historical data for one test zipcode
arima_model = sm.tsa.statespace.SARIMAX(train_df['value'], 
                                        enforce_stationarity=False, enforce_invertibility=False)

# fit the model and print results
fitted_model = arima_model.fit()

#### Printing the diagnostics of the model

In [ ]:
# plot model diagnostics
fitted_model.plot_diagnostics(figsize=(16, 8))
plt.show()

#### Evaluate the prediction and the test data

In [ ]:
# forecast the test data
forecast_values = fitted_model.predict(start=test_start_date, end=end_date, 
                                       typ='levels', dynamic=True).rename('predict')

In [ ]:
# plot historical and forecasted data
train_df['train'] = train_df['value']
train_df['train'].plot(legend=True,figsize=(12,6))
test_df['test'] = test_df['value']
test_df['test'].plot(legend=True)
forecast_values.plot(legend=True);


In [ ]:
# find the best fit model for the first test country
models = []
for country in top_confirmed_list[1:2]:
    result = auto_arima(df.loc[df['Country'] == country].value, 
                    start_p=1, start_q=1, max_p=3, max_q=3, m=12, start_P=0, seasonal=False, d=None, D=1, 
                    trace=False, error_action='ignore',suppress_warnings=True, stepwise=True)
    if result._is_seasonal():
        model = {'Country': country, 'model': 'SARIMAX', 'order': result.order,
                                     'seasonal_order': result.seasonal_order}
    else:
        model = {'Country': country, 'model': 'ARIMA', 'order': result.order, 
                                     'seasonal_order': None}
        
    models.append(model)
    
# convert models list into DataFrame for easy reading
model_df = pd.DataFrame(models, columns=['zipcode', 'model', 'order', 'seasonal_order'])
model_df

#### After having the model, we split data of this country into train/test dataset to evaluate the accurateness of the model

In [ ]:
second_country_data = df[df['Country']==country]
second_country_data.head()

In [ ]:
# get tail data of 3 year - test data set
second_country_data.tail(36)

In [ ]:
# split train/test data
#train_df = first_country_data.iloc[:train_days] #
train_df2 = first_country_data.loc[:split_date]
#test_df = first_country_data.iloc[split_date:]  #
test_start_date = split_date + timedelta(days=1) 
test_df2 = first_country_data.loc[test_start_date:]

In [ ]:
train_df2.tail()

In [ ]:
test_df2.head()

In [ ]:
train_df2.info()

In [ ]:
test_df2

In [ ]:
test_df2.info()

In [ ]:
# fit the best model with full historical data for one test zipcode
arima_model = sm.tsa.statespace.SARIMAX(train_df2['value'], 
                                        enforce_stationarity=False, enforce_invertibility=False)

# fit the model and print results
fitted_model = arima_model.fit()

#### Printing the diagnostics of the model

In [ ]:
# plot model diagnostics
fitted_model.plot_diagnostics(figsize=(16, 8))
plt.show()

#### Evaluate the prediction and the test data

In [ ]:
# forecast the test data
forecast_values = fitted_model.predict(start=test_start_date, end=end_date, 
                                       typ='levels', dynamic=True).rename('predict')

In [ ]:
# plot historical and forecasted data
train_df2['train'] = train_df2['value']
train_df2['train'].plot(legend=True,figsize=(12,6))
test_df2['test'] = test_df2['value']
test_df2['test'].plot(legend=True)
forecast_values.plot(legend=True);


### Calculate prediction error

In [ ]:
from statsmodels.tools.eval_measures import rmse

error = rmse(test_df2['value'], forecast_values)
print(f'RMSE: {error}')

In [ ]:
# calculate ROI, Profit
date_range = pd.date_range(start_date, periods=36, freq='MS')
forecast = pd.DataFrame(forecast_values, index=date_range[:])
value = train_df.iloc[-1]['value']
value_after_1_year = round(forecast.iloc[1*12-1]['predict'], 0)
value_after_2_year = round(forecast.iloc[2*12-1]['predict'], 0)
value_after_3_year = round(forecast.iloc[3*12-1]['predict'], 0)
forecast_data = {'Zipcode': zipcode, 'Current Value': value, 
                 'Value After 1 Year': value_after_1_year,
                 'Value After 2 Year': value_after_2_year,
                 'Value After 3 Year': value_after_3_year,
                 'Profit After 1 Year': value_after_1_year - value,
                 'Profit After 2 Year': value_after_2_year - value,
                 'Profit After 3 Year': value_after_3_year - value,
                 'ROI After 1 Year': round((value_after_1_year - value) / value, 2),
                 'ROI After 2 Year': round((value_after_2_year - value) / value, 2),
                 'ROI After 3 Year': round((value_after_3_year - value) / value, 2)}
forecast_data

# Step 5: Modelling and Forecasting The Future for All Zipcodes

#### Now we rewrite all the code for finding best models, fitting models and forecasting into reusable functions to perform forecasting for all the chosen zipcodes

In [ ]:
# function to find best fit model
def find_best_fit_models(dataframe):
    """
    This function use auto_arima to find the best fit model for each zipcode
    provided in the ``zipcodes`` list.
    
    Parameters:
    ------------
    dataframe   : the house value time series DataFrame with columns: ['zipcode', 'value']
    
    Return:
    ------------
    A DataFrame with following data columns: ['zipcode', 'model', 'order', 'seasonal_order']
    """
    # list to store best fit models
    models = []
    
    # zipcodes
    zipcodes = dataframe['zipcode'].unique()
    
    # loop through all zipcodes to find best model for each zipcode
    for zipcode in zipcodes:
        # call auto_arima to find best model
        result = auto_arima(dataframe.loc[dataframe['zipcode'] == zipcode].value, 
                            start_p=1, start_q=1, max_p=3, max_q=3, m=12, start_P=0, seasonal=True, d=None, D=1, 
                            trace=False, error_action='ignore',suppress_warnings=True, stepwise=True)
        
        # build a model dictionary and put into the returned list
        model = {'zipcode': zipcode, 'model': 'SARIMAX', 'order': result.order, 'seasonal_order': result.seasonal_order}
        models.append(model)

    # convert models list into DataFrame for easy reading
    model_df = pd.DataFrame(models, columns=['zipcode', 'model', 'order', 'seasonal_order'])
    
    return model_df

In [ ]:
# function to fit model and forecast future data
def fit_and_forecast(dataframe, model_df):
    """
    This function fit each model in ``model_df`` with the data from ``dataframe``.
    Then, use the fitted model to forecast into the future and calculate the forecasted ROI.
    
    Parameters:
    -----------
    dataframe   : the house value time series DataFrame with columns: ['zipcode', 'value']
    model_df    : the model DataFrame output from function ``find_best_fit_models``
    
    Return:
    -----------
    A DataFrame with forecasted data,
    And a DataFrame with forecasted ROI data
    """
    
    # forecast data to be returned
    ROIs = []
    forecasts = []
    
    # loop through all model in model_df
    for index, model in model_df.iterrows():                
        # fit the best model with full historical data for one test zipcode
        data = dataframe[dataframe['zipcode']==model['zipcode']]
        arima_model = sm.tsa.statespace.SARIMAX(data['value'], order=model['order'], 
                                                seasonal_order=model['seasonal_order'], 
                                                enforce_stationarity=False, enforce_invertibility=False)

        # fit the model and print results
        fitted_model = arima_model.fit()
    
        # last historical data point
        current_data = data.tail(1)
        current_data.reset_index(inplace=True)
        time = current_data.iloc[0]['time']
        zipcode = current_data.iloc[0]['zipcode']
        value = current_data.iloc[0]['value']
        
        # build forecast date range
        date_range = pd.date_range(time, periods=37, freq='MS') # first date is present
        date_range = date_range[1:] # remove the first date
        
        # forecast values of the future
        forecast_values = fitted_model.predict(start=date_range[0],
                  end=date_range[-1], typ='levels').rename('value')
        
        # build forecast dataframe
        forecast_df = pd.DataFrame(forecast_values, index=date_range[:])
        forecast_df['zipcode'] = zipcode
        forecasts.append(forecast_df)
        
        # calculate and build forecasted ROI dataframe
        value_after_1_year = round(forecast_df.iloc[1*12-1]['value'], 0)
        value_after_2_year = round(forecast_df.iloc[2*12-1]['value'], 0)
        value_after_3_year = round(forecast_df.iloc[3*12-1]['value'], 0)
        roi_data = {'Zipcode': zipcode, 
                         'Current Value': value, 
                         'Value After 1 Year':  value_after_1_year,
                         'Value After 2 Year':  value_after_2_year,
                         'Value After 3 Year':  value_after_3_year,
                         'Profit After 1 Year': value_after_1_year - value,
                         'Profit After 2 Year': value_after_2_year - value,
                         'Profit After 3 Year': value_after_3_year - value,
                         'ROI After 1 Year': round((value_after_1_year - value) / value, 2),
                         'ROI After 2 Year': round((value_after_2_year - value) / value, 2),
                         'ROI After 3 Year': round((value_after_3_year - value) / value, 2)}
        
        ROIs.append(roi_data)
    
    # convert models list into DataFrame for easy reading
    roi_df = pd.DataFrame(ROIs, columns=['Zipcode', 'Current Value', 
                           'Value After 1 Year', 'Value After 2 Year', 'Value After 3 Year',
                                'Profit After 1 Year', 'Profit After 2 Year', 'Profit After 3 Year',
                                'ROI After 1 Year', 'ROI After 2 Year', 'ROI After 3 Year'])
    
    # merge all forecasts into one DataFrame
    forecast_df = pd.concat(forecasts)
    
    return forecast_df, roi_df
    

##### Compare in 3 bar graphs, top30 of current_cases, top30 of current_deaths, top30 of current_recovered.
##### Each graph has 3 color bars show data of 3 current information

### STEP 8: USE ARIMA TO PREDICT TIME SERIES DATA
#### Since we don't have enough data to check if it has seasonal trend  therefore we don't have to check seasonal trend.

### STEP 9: DEEP LEARNING - CONVOLUTIONAL NEURAL NETWORK (CNN) - TO PREDICT

### STEP 10: 